# **베이스라인 모델**
* 베이스라인 모델의 제공 목적 
> - 첫째, 제공한 데이터세트를 활용한 머신러닝 Pipeline(분석절차)이 올바르게 동작하는지를 확인
> - 둘째, 제공한 데이터세트를 활용하여 분석 대상 기준 정의 및 제출 결과 파일 생성 방법 등에 참고

* 베이스라인 모델의 구조
> - 데이터 : 크게 train_bookmark라는 시청 이력 데이터와  train_Service라는 고객 상품별 재결제(해지) 이력 데이터
>> - 학습 데이터세트 분할 : train_service 70% & 30%로 분할하여 학습용과 테스트용으로 사용
> - 분석 알고리즘 : XGB 모델 
>> - 기본 설정 모델 제공 
* 베이스라인 모델은 쥬피터 노트북을 통한 Python으로 작성하여 제공
> - 주어진 베이스라인 모델은 예시 이며, 참여들은 자유롭게 분석 방법 및 항목 변경 가능


## **1. 학습 환경 세팅**

> ### 1) 코랩: 구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# 본인이 파일 업로드 한 폴더로 경로 지정

%cd /content/drive/My Drive/Colab Notebooks

> ### 2) Import Package 

In [ ]:
## 주요 라이브러리 import

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from dateutil.relativedelta import relativedelta
from datetime import *

## **2. 데이터 읽기**

In [ ]:
# 서비스 데이터 읽기 : 전체 69,708 rows
# 고객 상품(이용권)별 재결제(해지) 이력 정보
ds_service = "train_service.csv"
df_service = pd.read_csv(ds_service, parse_dates=['registerdate','enddate'], infer_datetime_format=True)

# 데이터에 대한 전반적인 정보를 표시 dataframe을 구성하는 행과 열의 크기, 컬럼명, 컬럼을 구성하는 값의 자료형 등을 출력
df_service.info()
# 데이터 샘플 3개 출력
df_service.sample(3)

In [ ]:
# 시청 이력(train_bookmark) 데이터 읽기 : 7,987,609 rows
ds_bookmark = "train_bookmark.csv"
df_bookmark = pd.read_csv(ds_bookmark, parse_dates=['dates'], infer_datetime_format=True)

# 데이터에 대한 전반적인 정보를 표시 dataframe을 구성하는 행과 열의 크기, 컬럼명, 컬럼을 구성하는 값의 자료형 등을 출력
df_bookmark.info()
# 데이터 샘플 3개 출력
df_bookmark.sample(3)

In [ ]:
# service 파일 컬럼별 unique values 확인
for column in df_service.columns.values.tolist():
    print(column)
    print(df_service[column].unique())
    print("")

In [ ]:
# bookmark 파일 컬럼별 unique values 확인 
for column in df_bookmark.columns.values.tolist():
    print(column)
    print(df_bookmark[column].unique())
    print("")

## **3. EDA (탐색적 데이터 분석)** 




> ### 1) 결측치 등 데이터 전처리 예시

In [ ]:
# 결과 중 결측치 비율을 보고 어느 컬럼을 삭제하고 어느 컬럼을 대체할지를 판단 
df_missing = df_service
np.sum(df_missing.isnull())
missing_number = df_missing.isnull().sum().sort_values(ascending=False)
missing_percentage = missing_number/len(df_missing)
missing_info = pd.concat([missing_number,missing_percentage],  axis=1, keys=['missing number','missing percentage'])
missing_info.head(50)

In [ ]:
# gender 값 및 분포 확인
print('gender = ', df_service.gender.unique().tolist())
df_service.gender.value_counts()

In [ ]:
# gender null 값을 N 으로 변경 후 확인
df_service['gender'] = df_service['gender'].fillna('N')
print('gender = ', sorted(df_service.gender.unique().tolist()))
df_service.gender.value_counts()

In [ ]:
# agegroup 값 및 분포 확인
print('agegroup = ', sorted(df_service.agegroup.unique().tolist()))
df_service.agegroup.value_counts()

In [ ]:
# agegroup 950 값을 0 으로 변환
df_service['agegroup'] = df_service['agegroup'].replace(950, 0)
print('agegroup = ', sorted(df_service.agegroup.unique().tolist()))
df_service.agegroup.value_counts()

In [ ]:
# pgamount 값 및 분포 확인
print('pgamount = ', sorted(df_service.pgamount.unique().tolist()))
df_service.pgamount.value_counts()

In [ ]:
# pgamount 금액 중에 달러로 결제된 것 원화로 변경 (pgamount 100원 미만인 건은 Appstore에서 달러 결제 건임)
df_service.loc[(df_service['pgamount'] <  100), 'pgamount'] = df_service['pgamount'] * 1120
print('pgamount = ', sorted(df_service.pgamount.unique().tolist()))
df_service.pgamount.value_counts()

In [ ]:
# 기타 컬럼들의 결측치 처리
df_service = df_service.fillna('X')

> ### 2) 자율 : Feature 생성 및 파생변수 생성은 자율로 추가: 예시 모델 제공  

In [ ]:
# 해지 예측 대상 서비스 추출
# 즉, 예측시점(가입일+3주)에 해지하지 않은 서비스 추출
df_svc_target = df_service[df_service.enddate.dt.date > (df_service.registerdate + pd.DateOffset(weeks=3)).dt.date]
print('df_svc_target = {0:,}'.format(df_svc_target.shape[0]), 'rows')

In [ ]:
# 변수 생성(Feature Extraction) (예시)
#   (1) 고객별 서비스 가입 이력 수
#   (2) 고객별 서비스 가입 이력 상품 수
#   (3) 고객별 시청 건수 (1시간 단위)
#   (4) 고객별 시청 총 시간
#   (5) 고객별 시청 평균 시간
#   (6) 고객별 시청 채널 수
#   (7) 고객별 시청 프로그램 수
#   (8) 고객별 시청 디바이스 수

# (1) 고객별 서비스 가입 이력 수
df_feature_1 = df_service.groupby(by='uno', as_index=False).registerdate.count()
df_feature_1.rename(columns={'registerdate':'REG_CNT'}, inplace=True)
print('REG_CNT = ', sorted(df_feature_1.REG_CNT.unique().tolist()))
#print('REG_CNT = ', df_feature_1.REG_CNT.value_counts())

# (2) 고객별 서비스 가입 이력 상품 수
df_feature_2 = df_service[['uno','productcode']]
df_feature_2 = df_feature_2.drop_duplicates() # 고객별 동일 상품 제거
df_feature_2 = df_feature_2.groupby(by='uno', as_index=False).productcode.count()
df_feature_2.rename(columns={'productcode':'PRD_CNT'}, inplace=True)
print('PRD_CNT = ', sorted(df_feature_2.PRD_CNT.unique().tolist()))
#print('PRD_CNT = ', df_feature_2.PRD_CNT.value_counts())

# (3) 고객별 시청 건수 (1시간 단위)
df_feature_3 = df_bookmark.groupby(by='uno', as_index=False).dates.count()
df_feature_3.rename(columns={'dates':'BM_CNT'}, inplace=True)

print('BM_CNT = ', sorted(df_feature_3.BM_CNT.unique().tolist()))
#print('BM_CNT = ', df_feature_3.BM_CNT.value_counts())

# (4) 고객별 시청 총 시간
df_feature_4 = df_bookmark.groupby(by='uno', as_index=False).viewtime.sum()
df_feature_4.rename(columns={'viewtime':'VT_TOT'}, inplace=True)
print('VT_TOT = ', sorted(df_feature_4.VT_TOT.unique().tolist()))
#print('VT_TOT = ', df_feature_4.VT_TOT.value_counts())

# (5) 고객별 시청 평균 시간
df_feature_5 = df_bookmark.groupby(by='uno', as_index=False).viewtime.mean()
df_feature_5.rename(columns={'viewtime':'VT_AVG'}, inplace=True)
print('VT_AVG = ', sorted(df_feature_5.VT_AVG.unique().tolist()))
#print('VT_AVG = ', df_feature_5.VT_AVG.value_counts())

# (6) 고객별 시청 채널 수
df_feature_6 = df_bookmark[['uno','channeltype']]
df_feature_6 = df_feature_6.drop_duplicates() # 고객별 동일 채널 제거
df_feature_6 = df_feature_6.groupby(by='uno', as_index=False).channeltype.count()
df_feature_6.rename(columns={'channeltype':'CH_CNT'}, inplace=True)
print('CH_CNT = ', sorted(df_feature_6.CH_CNT.unique().tolist()))
#print('CH_CNT = ', df_feature_6.CH_CNT.value_counts())

# (7) 고객별 시청 프로그램 수
df_feature_7 = df_bookmark[['uno','programid']]
df_feature_7 = df_feature_7.drop_duplicates() # 고객별 동일 프로그램 제거
df_feature_7 = df_feature_7.groupby(by='uno', as_index=False).programid.count()
df_feature_7.rename(columns={'programid':'PRG_CNT'}, inplace=True)
print('PRG_CNT = ', sorted(df_feature_7.PRG_CNT.unique().tolist()))
#print('PRG_CNT = ', df_feature_7.PRG_CNT.value_counts())

# (8) 고객별 시청 디바이스 수
df_feature_8 = df_bookmark[['uno','devicetype']]
df_feature_8 = df_feature_8.drop_duplicates() # 고객별 동일 프로그램 제거
df_feature_8 = df_feature_8.groupby(by='uno', as_index=False).devicetype.count()
df_feature_8.rename(columns={'devicetype':'DEV_CNT'}, inplace=True)
print('DEV_CNT = ', sorted(df_feature_8.DEV_CNT.unique().tolist()))
#print('DEV_CNT = ', df_feature_8.DEV_CNT.value_counts())

In [ ]:
print('df_feature_1 = ', df_feature_1.shape[0])
print('df_feature_2 = ', df_feature_2.shape[0])
print('df_feature_3 = ', df_feature_3.shape[0])
print('df_feature_4 = ', df_feature_4.shape[0])
print('df_feature_5 = ', df_feature_5.shape[0])
print('df_feature_6 = ', df_feature_6.shape[0])
print('df_feature_7 = ', df_feature_7.shape[0])
print('df_feature_8 = ', df_feature_8.shape[0])

In [ ]:
# 해지 예측 대상 서비스에 생성한 변수 연결
df_svc_target = pd.merge(df_svc_target, df_feature_1, on='uno', how='left')
df_svc_target = pd.merge(df_svc_target, df_feature_2, on='uno', how='left')
df_svc_target = pd.merge(df_svc_target, df_feature_3, on='uno', how='left')
df_svc_target = pd.merge(df_svc_target, df_feature_4, on='uno', how='left')
df_svc_target = pd.merge(df_svc_target, df_feature_5, on='uno', how='left')
df_svc_target = pd.merge(df_svc_target, df_feature_6, on='uno', how='left')
df_svc_target = pd.merge(df_svc_target, df_feature_7, on='uno', how='left')
df_svc_target = pd.merge(df_svc_target, df_feature_8, on='uno', how='left')

In [ ]:
# 추가 생성 컬럼들의 결측치 처리
df_svc_target = df_svc_target.fillna(0)
df_svc_target = df_svc_target.astype({'BM_CNT':'int', 'VT_TOT':'int', 'CH_CNT':'int', 'PRG_CNT':'int', 'DEV_CNT':'int'})

In [ ]:
# 최종 분석 대상 데이터
df_T = df_svc_target.copy()
df_T.info()
df_T.sample(5)

> ### 4) 상관관계/분포도 확인 

In [ ]:
# Encoding categorical variables to numeric ones

# Label 컬럼은 직접 변환
print('Repurchase = ', df_T.Repurchase.unique().tolist())
print(df_T.Repurchase.value_counts())
df_T['CHURN'] = np.where(df_T.Repurchase == 'X', 1, 0)   # 재결제('O') -> Churn Negative(0), 미결제('X') -> Churn Positive(1)
print('CHURN = ', df_T.CHURN.unique().tolist())
print(df_T.CHURN.value_counts())

# 이외 컬럼들은 일괄 자동 변환
col_lst = df_T.columns.values.tolist()
col_lst.remove('uno')
col_lst.remove('productcode')
for c in col_lst:
    if df_T[c].dtype == 'object':
        print(c)
        lbl = LabelEncoder()
        lbl.fit(list(df_T[c].values))
        df_T[c] = lbl.transform(df_T[c].values)
df_T.sample(5)

In [ ]:
# 기술 통계 현황 확인
df_T.describe()

In [ ]:
# Basic correlation plot to understand which features are correlated
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.figure(figsize=(18,10))
sns.heatmap(df_T.corr(), annot=True)   

In [ ]:
# This also helps in finding number of counts in each column
df_T.hist(figsize=(20,16))
plt.show()  # showing the charts of different columns

## **4. 모델 훈련**

> ### 1) Train/Test 데이터 설정 

In [ ]:
# 베이스라인 모델에서는 특정 월정액만 기준으로 진행 
# 실제 경연에서는 전체 월정액 대상
#df_T['productcode'].value_counts()
#df_T = df_T.loc[(df_T['productcode'] == 55)] # productcode = pk_1489

In [ ]:
# Train/Test 데이터 분리 
X = df_T.drop(["uno","registerdate","enddate","productcode","Repurchase","CHURN"], axis=1)
#X = df_T.drop(["registerdate","enddate","productcode","Repurchase","CHURN"], axis=1)
y = df_T["CHURN"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 111)

In [ ]:
print('Traing X     = ', len(X_train), ', Training Y   = ', len(y_train))
print('Validation X = ', len(X_test),  ', Validation Y = ', len(y_test))

> ### 2) 모델 생성 

In [ ]:
###
###        모델 학습 시 검증(평가) 지표 설정
###           ㅇ 본 경연의 평가 지표
###              - 1 순위 : F1
###              - 2 순위 : Accuracy
###

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier

#got the best parameters above
xgb = XGBClassifier(learning_rate = 0.3, max_depth = 3, n_estimators = 100,
                    eval_metric = 'aucpr', 
                    use_label_encoder=False, 
                    objective = 'binary:logistic', random_state = 1)
xgb.fit(X_train, y_train)

> ### 3) 학습 모델 예측력 검증

In [ ]:
y_pred = xgb.predict(X_test)

# printing the classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

> ### 4) 학습 모델 평가(Validation) 

In [ ]:
# confusion matrics to find precision and recall
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
###       예측 주제 관련 평가 지표 사용
###           ㅇ 예측 주제
###              - 이탈 즉, 2개월차 제결제하지 않음
###           ㅇ 이탈 여부 코딩
###              - 이탈 : 1 (= 재결제하지 않음)
###              - 유지 : 0 (= 재결제함)
###           ㅇ 평가 지표
###              - 이탈(1)에 대한 평가 지표로 Leaderboard 순위 부여

import sklearn.metrics as metrics

print('accuracy', metrics.accuracy_score(y_test,y_pred) )
print('precision', metrics.precision_score(y_test,y_pred) )
print('recall', metrics.recall_score(y_test,y_pred) )
print('f1', metrics.f1_score(y_test,y_pred) )
print(metrics.classification_report(y_test,y_pred))
print(metrics.confusion_matrix(y_test,y_pred))

## **5. 예측 결과 파일 제출**

In [ ]:
# Predict Input 파일 읽기 - 서비스 파일
ds_P_service = "predict_service.csv"
df_P_service = pd.read_csv(ds_P_service, parse_dates=['registerdate','enddate'], infer_datetime_format=True)

df_P_service.info()
#df_P_service.sample(5)

In [ ]:
# Predict Input 파일 읽기 - 북마크 파일
ds_P_bookmark = "predict_bookmark.csv"
df_P_bookmark = pd.read_csv(ds_P_bookmark, parse_dates=['dates'], infer_datetime_format=True)

df_P_bookmark.info()
#df_pbook.sample(5)

**학습 데이터 전처리 기준 동일 적용**

In [ ]:
# gender null 값을 N 으로 변경 후 확인
df_P_service['gender'] = df_P_service['gender'].fillna('N')

# agegroup 950 값을 0 으로 변환
df_P_service['agegroup'] = df_P_service['agegroup'].replace(950, 0)

# pgamount 금액 중에 달러로 결제된 것 원화로 변경 (pgamount 100원 미만인 건은 Appstore에서 달러 결제 건임)
df_P_service.loc[(df_P_service['pgamount'] <  100), 'pgamount'] = df_P_service['pgamount'] * 1120

# 기타 컬럼들의 결측치 처리
df_P_service = df_P_service.fillna('X')

# 해지 예측 대상 서비스 추출
# 즉, 예측시점(가입일+3주)에 해지하지 않은 서비스 추출
df_P_svc_target = df_P_service[df_P_service.enddate >= df_P_service.registerdate + pd.DateOffset(weeks=3)]

# 변수 생성(Feature Extraction) (예시)
# (1) 고객별 서비스 가입 이력 수
df_P_feature_1 = df_P_service.groupby(by='uno', as_index=False).registerdate.count()
df_P_feature_1.rename(columns={'registerdate':'REG_CNT'}, inplace=True)

# (2) 고객별 서비스 가입 이력 상품 수
df_P_feature_2 = df_P_service[['uno','productcode']]
df_P_feature_2 = df_P_feature_2.drop_duplicates() # 고객별 동일 상품 제거
df_P_feature_2 = df_P_feature_2.groupby(by='uno', as_index=False).productcode.count()
df_P_feature_2.rename(columns={'productcode':'PRD_CNT'}, inplace=True)

# (3) 고객별 시청 건수 (1시간 단위)
df_P_feature_3 = df_P_bookmark.groupby(by='uno', as_index=False).dates.count()
df_P_feature_3.rename(columns={'dates':'BM_CNT'}, inplace=True)

# (4) 고객별 시청 총 시간
df_P_feature_4 = df_P_bookmark.groupby(by='uno', as_index=False).viewtime.sum()
df_P_feature_4.rename(columns={'viewtime':'VT_TOT'}, inplace=True)

# (5) 고객별 시청 평균 시간
df_P_feature_5 = df_P_bookmark.groupby(by='uno', as_index=False).viewtime.mean()
df_P_feature_5.rename(columns={'viewtime':'VT_AVG'}, inplace=True)

# (6) 고객별 시청 채널 수
df_P_feature_6 = df_P_bookmark[['uno','channeltype']]
df_P_feature_6 = df_P_feature_6.drop_duplicates() # 고객별 동일 채널 제거
df_P_feature_6 = df_P_feature_6.groupby(by='uno', as_index=False).channeltype.count()
df_P_feature_6.rename(columns={'channeltype':'CH_CNT'}, inplace=True)

# (7) 고객별 시청 프로그램 수
df_P_feature_7 = df_P_bookmark[['uno','programid']]
df_P_feature_7 = df_P_feature_7.drop_duplicates() # 고객별 동일 프로그램 제거
df_P_feature_7 = df_P_feature_7.groupby(by='uno', as_index=False).programid.count()
df_P_feature_7.rename(columns={'programid':'PRG_CNT'}, inplace=True)

# (8) 고객별 시청 디바이스 수
df_P_feature_8 = df_P_bookmark[['uno','devicetype']]
df_P_feature_8 = df_P_feature_8.drop_duplicates() # 고객별 동일 프로그램 제거
df_P_feature_8 = df_P_feature_8.groupby(by='uno', as_index=False).devicetype.count()
df_P_feature_8.rename(columns={'devicetype':'DEV_CNT'}, inplace=True)

# 해지 예측 대상 서비스에 생성한 변수 연결
df_P_svc_target = pd.merge(df_P_svc_target, df_P_feature_1, on='uno', how='left')
df_P_svc_target = pd.merge(df_P_svc_target, df_P_feature_2, on='uno', how='left')
df_P_svc_target = pd.merge(df_P_svc_target, df_P_feature_3, on='uno', how='left')
df_P_svc_target = pd.merge(df_P_svc_target, df_P_feature_4, on='uno', how='left')
df_P_svc_target = pd.merge(df_P_svc_target, df_P_feature_5, on='uno', how='left')
df_P_svc_target = pd.merge(df_P_svc_target, df_P_feature_6, on='uno', how='left')
df_P_svc_target = pd.merge(df_P_svc_target, df_P_feature_7, on='uno', how='left')
df_P_svc_target = pd.merge(df_P_svc_target, df_P_feature_8, on='uno', how='left')

# 추가 생성 컬럼들의 결측치 처리
df_svc_target = df_svc_target.fillna(0)
df_svc_target = df_svc_target.astype({'BM_CNT':'int', 'VT_TOT':'int', 'CH_CNT':'int', 'PRG_CNT':'int', 'DEV_CNT':'int'})

df_P_svc_target.info()

In [ ]:
# 최종 분석 대상 데이터
df_P = df_P_svc_target.copy()

# Encoding categorical variables to numeric ones
col_lst = df_P.columns.values.tolist()
col_lst.remove('uno')
col_lst.remove('productcode')
for c in col_lst:
    if df_P[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(df_P[c].values))
        df_P[c] = lbl.transform(df_P[c].values)

df_P.info()

In [ ]:
# 예측 모델 입력을 위한 최종 데이터
X_predict = df_P.drop(["uno","registerdate","enddate","productcode","Repurchase"], axis=1)

**예측 모델에 의핸 해지 예측**

In [ ]:
# 예측 모델에 최종 데이터 입력
y_pred = xgb.predict(X_predict)

In [ ]:
# 데이터 건수 확인
print('예측 대상 건수 = ', len(X_predict), ', 예측 결과 건수 = ', len(y_pred))

**예측 결과 답안지 제출**

In [2]:
# 결과 제출 답안지 불러오기
ds_sheet = "CDS_submission.csv"
df_sheet = pd.read_csv(ds_sheet)
df_sheet.drop('CHURN', axis=1, inplace=True)
df_sheet.info()

# 답안지에 답안 표기
df_result = df_P.loc[:,('uno','registerdate','productcode')]
df_result['KEY']   = df_result['uno'] + '|' + df_result['registerdate'].dt.strftime('%y-%m-%d %I:%M:%S') + '|' + df_result['productcode']   # 판다스 strftime()
df_result['CHURN'] = pd.DataFrame(y_pred)
df_result = df_result.loc[:,('KEY','CHURN')]
df_answer_sheet = pd.merge(df_sheet, df_result, on='KEY', how='left')
df_answer_sheet.info()

NameError: name 'pd' is not defined

In [ ]:
# 답안지 제출 파일 생성하기
ds_answer_sheet = "CDS_submission_팀명_차수.csv"
df_answer_sheet.to_csv(ds_answer_sheet, index=False, encoding='utf8')

# End